# Not sure if this file is up to date - for operational usage go to Experiment_Automation.ipynb !!


# Test Environment

In [3]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
#import Models.models as models
#import Models.LSTM.models_LSTM as models
import script_utils as mL
#from Experiment_Class import Experiment
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import os
import time

NDBC = mL.NDBC_lib
ERA5 = mL.ERA5_lib
Models = mL.Models
DP = mL.DataProcessor
Experiment = mL.Experiment

# Get Data

In [4]:
data_directory = os.path.join(os.getcwd(), f'data/datasets/type_B')

def build_UI():

    # Select Model-------------------------------------------------------------------------------
    global datafile_widget
    datafile_list = os.listdir(data_directory)
    datafile_widget = widgets.Select(
        options=datafile_list,
        value=datafile_list[0],
        # rows=10,
        description='Datafile:',
        disabled=False
    )

    #STATIONARY_SHIFT -----------------------------------------------------------------------
    global stationaryShift_widget
    stationaryShift_widget = widgets.BoundedIntText(
        value=1,
        min=0,
        max=10,
        step=1,
        description='',
        disabled=False,
    )


    # Select Model-------------------------------------------------------------------------------
    global models_widget
    models_list = list(Models.model_dictionary.keys())
    models_widget = widgets.Select(
        options=models_list,
        value=models_list[0],
        # rows=10,
        description='Model:',
        disabled=False
    )


    #ALPHA:-------------------------------------------------------------------------------
    # create a FloatSlider widget for a value between 0 and 1
    global alpha_slider
    alpha_slider = widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.01,
        description='',
        readout_format='.2f',
        orientation='horizontal',
        layout={'width': '500px'}
    )


    left_box = widgets.VBox([
        datafile_widget,
        widgets.HTML(value="<p>Station Shift:</p>"),
        stationaryShift_widget
        ],
    )


    right_box = widgets.VBox([
        models_widget
    ])


    display(widgets.HBox([
        left_box,
        right_box,
    ]))
    print("Alpha: ")
    display(alpha_slider)


build_UI()

Alpha: 


FloatSlider(value=0.5, layout=Layout(width='500px'), max=1.0, step=0.01)

In [13]:
DATAFILE = datafile_widget.value
#print(DATAFILE)
# STATIONARY_SHIFT = stationaryShift_widget.value
# MODEL_NAME = models_widget.value
# ALPHA = alpha_slider.value

## Optional: use hardcoded variables instead

In [14]:
#DATAFILE = "dataset_GOM_1_B_B.pickle"
STATIONARY_SHIFT = 1
MODEL_NAME = "LSTM"
ALPHA = 0

In [15]:
print(f"Datafile: {DATAFILE}")
print(f"Stationary Shift: {STATIONARY_SHIFT}")
print(f"Model: {MODEL_NAME}")
print(f"Alpha: {ALPHA}")

Datafile: GOM_buoys_since2002_nan50_SSUM.pickle
Stationary Shift: 1
Model: LSTM
Alpha: 0


In [16]:
with open(f'data/datasets/type_B/{DATAFILE}', 'rb') as f:
    # load the object from the file using pickle.load()
    dataset = pickle.load(f)

print("files:", dataset["files"])
print("nan_threshold", dataset["nan_threshold"])
print("features", dataset["features"])
print("add_era5", dataset["add_era5"])
print("num_test_hours", dataset["num_test_hours"])

train = dataset["data_train"]
test = dataset["data_test"]

display(train)
display(test)

files: [['42001', '2002'], ['42002', '2002'], ['42003', '2002'], ['42007', '2002'], ['42019', '2002'], ['42020', '2002'], ['42035', '2002'], ['42036', '2002'], ['42039', '2002'], ['42040', '2002'], ['42041', '2002'], ['42001', '2003'], ['42002', '2003'], ['42003', '2003'], ['42007', '2003'], ['42019', '2003'], ['42020', '2003'], ['42035', '2003'], ['42036', '2003'], ['42039', '2003'], ['42040', '2003'], ['42041', '2003'], ['42001', '2004'], ['42002', '2004'], ['42003', '2004'], ['42007', '2004'], ['42019', '2004'], ['42020', '2004'], ['42035', '2004'], ['42036', '2004'], ['42038', '2004'], ['42039', '2004'], ['42040', '2004'], ['42041', '2004'], ['42001', '2005'], ['42002', '2005'], ['42003', '2005'], ['42007', '2005'], ['42019', '2005'], ['42020', '2005'], ['42035', '2005'], ['42036', '2005'], ['42038', '2005'], ['42039', '2005'], ['42040', '2005'], ['42041', '2005'], ['42055', '2005'], ['42001', '2006'], ['42002', '2006'], ['42003', '2006'], ['42007', '2006'], ['42019', '2006'], ['42

""
timestamp
2002-01-01 00:00:00
2002-01-01 01:00:00
2002-01-01 02:00:00
2002-01-01 03:00:00
2002-01-01 04:00:00
...
2022-12-30 17:00:00
2022-12-30 18:00:00
2022-12-30 19:00:00


""
timestamp
2022-12-30 20:00:00
2022-12-30 21:00:00
2022-12-30 22:00:00
2022-12-30 23:00:00
2022-12-31 00:00:00
2022-12-31 01:00:00
2022-12-31 02:00:00
2022-12-31 03:00:00
2022-12-31 04:00:00


# Data Processing

## 1. Make data stationary

In [46]:
STATIONARY = True #Set Flag for report
train_stationary = DP.data_to_stationary(train, n = STATIONARY_SHIFT)
test_stationary = DP.data_to_stationary(test, n = STATIONARY_SHIFT)

print(f"TRAIN: shape: {train_stationary.shape}")
display(train_stationary.head())

print(f"TEST: shape: {test_stationary.shape}")
display(test_stationary.head())

TRAIN: shape: (832843, 24)


,WDIR(t-3),WSPD(t-3),PRES(t-3),WDIR_ERA5(t-3),WSPD_ERA5(t-3),PRES_ERA5(t-3),WDIR(t-2),WSPD(t-2),PRES(t-2),WDIR_ERA5(t-2),...,PRES(t-1),WDIR_ERA5(t-1),WSPD_ERA5(t-1),PRES_ERA5(t-1),WDIR(t),WSPD(t),PRES(t),WDIR_ERA5(t),WSPD_ERA5(t),PRES_ERA5(t)
2002-01-01 05:00:00,1.0,0.1,0.1,-2.838651,-0.265851,0.107580,2.0,-0.3,0.0,-2.869363,...,-0.1,0.210819,-0.429966,0.541160,-1.0,0.6,-0.6,1.433525,-0.446340,-0.279708
2002-01-01 06:00:00,2.0,-0.3,0.0,-2.869363,-0.605774,-0.344908,1.0,-0.1,-0.1,0.210819,...,-0.6,1.433525,-0.446340,-0.279708,3.0,-0.1,-0.3,-0.611968,0.416342,0.069112
2002-01-01 07:00:00,1.0,-0.1,-0.1,0.210819,-0.429966,0.541160,-1.0,0.6,-0.6,1.433525,...,-0.3,-0.611968,0.416342,0.069112,-2.0,0.1,0.1,0.677998,0.504338,-0.183212
2002-01-01 13:00:00,8.0,1.2,-1.4,-1.134972,1.175009,-1.161864,11.0,0.4,-0.8,-10.320926,...,-1.3,-18.025845,1.599584,-0.987128,20.0,0.6,-1.1,-28.439841,0.753363,-0.605056
2002-01-01 14:00:00,2.0,-0.2,0.0,-7.752430,0.408429,0.158436,7.0,0.7,-0.8,-8.316887,...,0.3,-9.735999,-0.341883,0.198860,-1.0,0.6,0.4,-5.044216,-0.446621,0.691772


TEST: shape: (24, 24)


,WDIR(t-3),WSPD(t-3),PRES(t-3),WSPD_ERA5(t-3),WDIR_ERA5(t-3),PRES_ERA5(t-3),WDIR(t-2),WSPD(t-2),PRES(t-2),WSPD_ERA5(t-2),...,PRES(t-1),WSPD_ERA5(t-1),WDIR_ERA5(t-1),PRES_ERA5(t-1),WDIR(t),WSPD(t),PRES(t),WSPD_ERA5(t),WDIR_ERA5(t),PRES_ERA5(t)
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-12-31 00:00:00,4.0,-1.2,0.0,-1.676501,-9.228674,0.027344,7.0,0.0,0.4,-0.872666,...,0.4,-1.363622,-13.103055,0.241086,4.0,-0.9,0.6,-0.243606,-8.966873,0.546872
2022-12-31 01:00:00,7.0,0.0,0.4,-0.872666,15.261675,0.148272,0.0,-1.2,0.4,-1.363622,...,0.6,-0.243606,-8.966873,0.546872,14.0,0.6,0.3,0.530232,0.785140,0.788342
2022-12-31 02:00:00,0.0,-1.2,0.4,-1.363622,-13.103055,0.241086,4.0,-0.9,0.6,-0.243606,...,0.3,0.530232,0.785140,0.788342,-34.0,-3.3,0.7,0.252866,4.558071,0.613498
2022-12-31 03:00:00,4.0,-0.9,0.6,-0.243606,-8.966873,0.546872,14.0,0.6,0.3,0.530232,...,0.7,0.252866,4.558071,0.613498,-116.0,3.7,0.3,0.622488,-2.838479,0.403992
2022-12-31 04:00:00,14.0,0.6,0.3,0.530232,0.785140,0.788342,-34.0,-3.3,0.7,0.252866,...,0.3,0.622488,-2.838479,0.403992,2.0,2.5,0.4,0.037617,-7.927598,0.105523


## 2. Transform to supervised problem

In [47]:
#Is already supervised!

## train test split

In [48]:
train_X, train_y, _, _ = DP.train_test_split(train_stationary, -len(train_stationary))
_, _, test_X, test_y = DP.train_test_split(test_stationary, len(test_stationary))
print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

Shapes:  (832843, 3, 6) (832843, 6) (24, 3, 6) (24, 6)


## Normalize the data

In [49]:
# Scale the data
#NORMALIZED = True   #set flag for report

# train_X_scaled, train_y_scaled, test_X_scaled, test_y_scaled, SCALER = DP.scale_data(train_X,
#                                                                                      train_y,
#                                                                                      test_X,
#                                                                                      test_y)

train_X_scaled, train_y_scaled, test_X_scaled, test_y_scaled = train_X, train_y, test_X, test_y #just to exclude scaling for now!

---
# Select Model

In [50]:
#model = models.lstm_0(train_X_scaled,train_y_scaled)
start_time = time.time()
model = Models.get_model(MODEL_NAME, train_X, train_y, ALPHA)
TRAINING_TIME = time.time() - start_time
model.summary()

Epoch 1/100
11712/11712 [==============================] - 98s 8ms/step - loss: 1107.1050 - val_loss: 1061.1262
Epoch 2/100
11712/11712 [==============================] - 103s 9ms/step - loss: 1050.7704 - val_loss: 1026.7465
Epoch 3/100
11712/11712 [==============================] - 92s 8ms/step - loss: 1032.4683 - val_loss: 1018.8115
Epoch 4/100
11712/11712 [==============================] - 92s 8ms/step - loss: 1024.4038 - val_loss: 1010.4485
Epoch 5/100
11712/11712 [==============================] - 85s 7ms/step - loss: 1019.1964 - val_loss: 1007.7982
Epoch 6/100
11712/11712 [==============================] - 85s 7ms/step - loss: 1016.2034 - val_loss: 1004.0020
Epoch 7/100
11712/11712 [==============================] - 86s 7ms/step - loss: 1013.0759 - val_loss: 1002.7194
Epoch 8/100
11712/11712 [==============================] - 88s 7ms/step - loss: 1010.8411 - val_loss: 1003.3090
Epoch 9/100
11712/11712 [==============================] - 85s 7ms/step - loss: 1010.0564 - val_loss: 1

----
# One-Shot Forecasting

In [51]:
_ = model.predict(train_X_scaled, batch_size=1)

832843/832843 [==============================] - 787s 944us/step


In [52]:
output_cols  = test.loc[:, ~test.columns.str.contains('t-')].columns
#output_cols

In [53]:
yhat = model.predict(test_X_scaled)
#yhat

1/1 [==============================] - 1s 903ms/step


In [54]:
#yhat_unscaled = DP.invert_scaling(yhat, SCALER)
yhat_unscaled = yhat # since test_X_scaled is actually not scaled - lol :D
#yhat_unscaled

In [55]:
yhat_unscaled_df = pd.DataFrame(yhat_unscaled)
yhat_unscaled_df.columns = [name + "_pred" for name in output_cols]
yhat_unscaled_df.set_index(test.tail(len(yhat)).index, inplace=True)
#yhat_unscaled_df

In [56]:
evaluation_1 = test.loc[:, ~test.columns.str.contains('t-')]    #ground truth: just y, without X
#evaluation_1

In [57]:
for col in evaluation_1.columns:
    shifted = evaluation_1[col].shift(STATIONARY_SHIFT)
    evaluation_1[f"{col}_pred"]= shifted + yhat_unscaled_df[f"{col}_pred"]

/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_63573/795191291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_1[f"{col}_pred"]= shifted + yhat_unscaled_df[f"{col}_pred"]
/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_63573/795191291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_1[f"{col}_pred"]= shifted + yhat_unscaled_df[f"{col}_pred"]
/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_63573/795191291.py:3: SettingWithCopyWarning: 
A value is

In [58]:
evaluation_1 = evaluation_1.iloc[STATIONARY_SHIFT:]  # remove first n entries since there is no delta value for them
evaluation_1

,WDIR(t),WSPD(t),PRES(t),WSPD_ERA5(t),WDIR_ERA5(t),PRES_ERA5(t),WDIR(t)_pred,WSPD(t)_pred,PRES(t)_pred,WSPD_ERA5(t)_pred,WDIR_ERA5(t)_pred,PRES_ERA5(t)_pred
timestamp,,,,,,,,,,,,
2022-12-31 00:00:00,189.0,5.0,1013.5,5.057694,66.884523,1014.403047,188.511487,5.875141,1012.892656,6.955716,76.511123,1014.329291
2022-12-31 01:00:00,203.0,5.6,1013.8,5.587926,67.669663,1015.191389,195.363029,4.958570,1013.513761,6.719800,67.554375,1014.860011
2022-12-31 02:00:00,169.0,2.3,1014.5,5.840792,72.227735,1015.804887,197.093222,5.541854,1013.854648,7.262590,68.348238,1015.780335
2022-12-31 03:00:00,53.0,6.0,1014.8,6.463279,69.389256,1016.208879,172.070677,2.318695,1014.477424,7.387517,72.797478,1016.257625
2022-12-31 04:00:00,55.0,8.5,1015.2,6.500897,61.461657,1016.314402,103.891296,6.325666,1014.832072,7.675694,69.895695,1015.870008
2022-12-31 05:00:00,72.0,7.6,1014.9,6.541557,53.576686,1016.235452,79.556767,8.651113,1015.207445,8.084803,61.961520,1016.483853
2022-12-31 06:00:00,77.0,7.2,1014.4,6.465415,56.308269,1015.708607,87.234285,7.674629,1014.888717,8.218962,54.192594,1016.579179
2022-12-31 07:00:00,97.0,7.8,1014.1,6.487022,61.932861,1015.506803,71.169741,7.161522,1014.452869,8.134612,56.997164,1016.285588
2022-12-31 08:00:00,148.0,3.3,1013.9,6.690521,65.444207,1015.204484,94.172756,7.754755,1014.125072,8.124779,62.580805,1016.055912


# Correct wind direction (modulo 360)

In [59]:
# Get the list of columns starting with "WDIR"
wdir_columns = [col for col in evaluation_1.columns if col.startswith("WDIR")]

# Modify the values in the selected columns
evaluation_1[wdir_columns] = evaluation_1[wdir_columns] % 360

evaluation_1

,WDIR(t),WSPD(t),PRES(t),WSPD_ERA5(t),WDIR_ERA5(t),PRES_ERA5(t),WDIR(t)_pred,WSPD(t)_pred,PRES(t)_pred,WSPD_ERA5(t)_pred,WDIR_ERA5(t)_pred,PRES_ERA5(t)_pred
timestamp,,,,,,,,,,,,
2022-12-31 00:00:00,189.0,5.0,1013.5,5.057694,66.884523,1014.403047,188.511487,5.875141,1012.892656,6.955716,76.511123,1014.329291
2022-12-31 01:00:00,203.0,5.6,1013.8,5.587926,67.669663,1015.191389,195.363029,4.958570,1013.513761,6.719800,67.554375,1014.860011
2022-12-31 02:00:00,169.0,2.3,1014.5,5.840792,72.227735,1015.804887,197.093222,5.541854,1013.854648,7.262590,68.348238,1015.780335
2022-12-31 03:00:00,53.0,6.0,1014.8,6.463279,69.389256,1016.208879,172.070677,2.318695,1014.477424,7.387517,72.797478,1016.257625
2022-12-31 04:00:00,55.0,8.5,1015.2,6.500897,61.461657,1016.314402,103.891296,6.325666,1014.832072,7.675694,69.895695,1015.870008
2022-12-31 05:00:00,72.0,7.6,1014.9,6.541557,53.576686,1016.235452,79.556767,8.651113,1015.207445,8.084803,61.961520,1016.483853
2022-12-31 06:00:00,77.0,7.2,1014.4,6.465415,56.308269,1015.708607,87.234285,7.674629,1014.888717,8.218962,54.192594,1016.579179
2022-12-31 07:00:00,97.0,7.8,1014.1,6.487022,61.932861,1015.506803,71.169741,7.161522,1014.452869,8.134612,56.997164,1016.285588
2022-12-31 08:00:00,148.0,3.3,1013.9,6.690521,65.444207,1015.204484,94.172756,7.754755,1014.125072,8.124779,62.580805,1016.055912


In [60]:
# wtmp_true = [col for col in evaluation_1.columns if col.startswith("WTMP")][0]
#
# mae = mean_absolute_error(evaluation_1[wtmp_true], evaluation_1[f"{wtmp_true}_pred"])
# mse = mean_squared_error(evaluation_1[wtmp_true], evaluation_1[f"{wtmp_true}_pred"])
# print('MAE: ', mae)
# print('MSE: ', mse)

In [61]:
#evaluation_1.plot(kind='line')

---
# SAVE

In [62]:
# create a text input widget for username
# filename_widget = widgets.Text(
#     value='',
#     placeholder='Enter filename',
#     description='Filename:',
#     disabled=False
# )
# # add '.csv' to the description
# extension_label = widgets.Label('.pickle')
#
# # display the widget
# display(widgets.HBox([filename_widget, extension_label]))
#
# print("Please also check if the reports description needs to be changed!")

In [63]:
report_description="Executed with Test_ML_Models_B script!"

In [64]:
#filename = filename_widget.value

filename = "report_B_LSTM_1.0"


if filename == "":
    print("Enter a valid filename!")

else:
    #Save Data About executed Test:

    #Convert model summary to string
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)

    report = Experiment(
        name=filename,
        description=report_description,

        stations = dataset["files"],
        years = ["not available"],
        nan_threshold=dataset["nan_threshold"],
        features=dataset["features"],
        era5=dataset["add_era5"],

        stationary_shift=STATIONARY_SHIFT,
        # lag=1,
        n_test_hours=dataset["num_test_hours"],

        #stationary=STATIONARY,
        scaler=None, #SCALER,

        model_name = MODEL_NAME,
        model_summary=model_summary,
        training_time = TRAINING_TIME,

        one_shot_forecast = evaluation_1,
        recursive_forecast = None
    )


    # open a file for writing in binary mode
    filepath = f'data/reports/{filename}.pickle'
    with open(filepath, 'wb') as f:
        # write the object to the file using pickle.dump()
        pickle.dump(report, f)
        print("File successfully saved:")
        print(filepath)

File successfully saved:
data/reports/report_B_LSTM_1.0.pickle


In [65]:
print("DONE!")

DONE!
